In [1]:
"""本文件代码主要尝试是融合以下内容,然后做特征选择
1、Mean Encoding
2、重要的cate feature groupby_max_min_median_std_mean
3、重要特征count
4、重要业务特征crosses
"""

'本文件代码主要尝试是融合以下内容,然后做特征选择\n1、Mean Encoding\n2、重要的cate feature groupby_max_min_median_std_mean\n3、重要特征count\n4、重要业务特征crosses\n'

In [2]:
import warnings
import os
import gc
import tqdm
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
from itertools import product
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.feature_selection import VarianceThreshold,chi2
from sklearn.feature_selection import mutual_info_classif,SelectKBest
from scipy.stats import pearsonr
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split



warnings.filterwarnings("ignore")

In [3]:
sys.version

'3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 05:35:01) [MSC v.1916 64 bit (AMD64)]'

In [4]:
train_X = pd.read_csv("./train.csv")
test_X = pd.read_csv("./testA.csv")

In [5]:
class MeanEncoder:
    def __init__(self, categorical_features, n_splits=5, target_type='classification', prior_weight_func=None):
        """
        :param categorical_features: list of str, the name of the categorical columns to encode

        :param n_splits: the number of splits used in mean encoding

        :param target_type: str, 'regression' or 'classification'

        :param prior_weight_func:
        a function that takes in the number of observations, and outputs prior weight
        when a dict is passed, the default exponential decay function will be used:
        k: the number of observations needed for the posterior to be weighted equally as the prior
        f: larger f --> smaller slope
        """

        self.categorical_features = categorical_features
        self.n_splits = n_splits
        self.learned_stats = {}

        if target_type == 'classification':
            self.target_type = target_type
            self.target_values = []
        else:
            self.target_type = 'regression'
            self.target_values = None

        if isinstance(prior_weight_func, dict):
            self.prior_weight_func = eval('lambda x: 1 / (1 + np.exp((x - k) / f))', dict(prior_weight_func, np=np))
        elif callable(prior_weight_func):
            self.prior_weight_func = prior_weight_func
        else:
            self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))

    @staticmethod
    def mean_encode_subroutine(X_train, y_train, X_test, variable, target, prior_weight_func):
        X_train = X_train[[variable]].copy()
        X_test = X_test[[variable]].copy()

        if target is not None:
            nf_name = '{}_pred_{}'.format(variable, target)
            X_train['pred_temp'] = (y_train == target).astype(int)  # classification
        else:
            nf_name = '{}_pred'.format(variable)
            X_train['pred_temp'] = y_train  # regression
        prior = X_train['pred_temp'].mean()

        col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg([('mean', 'mean'), ('beta', 'size')])
        col_avg_y['beta'] = prior_weight_func(col_avg_y['beta'])
        col_avg_y[nf_name] = col_avg_y['beta'] * prior + (1 - col_avg_y['beta']) * col_avg_y['mean']
        col_avg_y.drop(['beta', 'mean'], axis=1, inplace=True)

        nf_train = X_train.join(col_avg_y, on=variable)[nf_name].values
        nf_test = X_test.join(col_avg_y, on=variable).fillna(prior, inplace=False)[nf_name].values

        return nf_train, nf_test, prior, col_avg_y

    def fit_transform(self, X, y):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :param y: pandas Series or numpy array, n_samples
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
        if self.target_type == 'classification':
            skf = StratifiedKFold(self.n_splits)
        else:
            skf = KFold(self.n_splits)

        if self.target_type == 'classification':
            self.target_values = sorted(set(y))
            self.learned_stats = {'{}_pred_{}'.format(variable, target): [] for variable, target in
                product(self.categorical_features, self.target_values)}
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, target, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        else:
            self.learned_stats = {'{}_pred'.format(variable): [] for variable in self.categorical_features}
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, None, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        return X_new

    def transform(self, X):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()

        if self.target_type == 'classification':
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
        else:
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits

        return X_new


## 处理缺失值

In [6]:
#含缺失值的列名
missing_cols = train_X.columns[train_X.isnull().sum() > 0].to_list()

In [7]:
missing_cols

['employmentTitle',
 'employmentLength',
 'postCode',
 'dti',
 'pubRecBankruptcies',
 'revolUtil',
 'title',
 'n0',
 'n1',
 'n2',
 'n3',
 'n4',
 'n5',
 'n6',
 'n7',
 'n8',
 'n9',
 'n10',
 'n11',
 'n12',
 'n13',
 'n14']

In [8]:
for col in missing_cols:
    if train_X[col].dtype != 'object':
        imputer = SimpleImputer(strategy='constant', fill_value=-999)
        train_X[col] = imputer.fit_transform(train_X[[col]])
        del imputer
        gc.collect()

In [9]:
gc.collect()

21

In [10]:
#employmentLength 类型为object，先将其编码，再填充缺失值
label_enc = LabelEncoder()
train_X['employmentLength'] = label_enc.fit_transform(train_X[['employmentLength']])
test_X['employmentLength'] = label_enc.transform(test_X[['employmentLength']])


imputer = SimpleImputer(strategy='constant',fill_value=-999)
train_X['employmentLength'] = imputer.fit_transform(train_X[['employmentLength']])
test_X['employmentLength'] = imputer.transform(test_X[['employmentLength']])

In [11]:
train_X.isnull().sum()

id                    0
loanAmnt              0
term                  0
interestRate          0
installment           0
grade                 0
subGrade              0
employmentTitle       0
employmentLength      0
homeOwnership         0
annualIncome          0
verificationStatus    0
issueDate             0
isDefault             0
purpose               0
postCode              0
regionCode            0
dti                   0
delinquency_2years    0
ficoRangeLow          0
ficoRangeHigh         0
openAcc               0
pubRec                0
pubRecBankruptcies    0
revolBal              0
revolUtil             0
totalAcc              0
initialListStatus     0
applicationType       0
earliesCreditLine     0
title                 0
policyCode            0
n0                    0
n1                    0
n2                    0
n3                    0
n4                    0
n5                    0
n6                    0
n7                    0
n8                    0
n9              

In [12]:
train_X['employmentLength'].value_counts()

1     262753
2      72358
10     64237
3      64152
0      52489
5      50102
4      47985
11     46799
6      37254
8      36192
7      35407
9      30272
Name: employmentLength, dtype: int64

## 部分特征编码

### 先将object类型的特征进行编码

In [13]:
object_cols = train_X.columns[pd.Series(train_X.columns).apply(lambda x: True if train_X[x].dtype == 'object' else False)]
object_cols = list(object_cols)
object_cols

['grade', 'subGrade', 'issueDate', 'earliesCreditLine']

In [14]:
# 根据grade 和subGrade 的是数据类型，将其进行序列编码
label_enc = LabelEncoder()
for col in ['grade','subGrade']:
    train_X[col] = label_enc.fit_transform(train_X[[col]])
    test_X[col] = label_enc.transform(test_X[[col]])

In [15]:
#'issueDate', 'earliesCreditLine'属于日期类型特征，将其年月日提取出来

#earliesCreditLine
train_X['earliesCreditLine_year'] = pd.DatetimeIndex(train_X['earliesCreditLine']).year
train_X['earliesCreditLine_month'] = pd.DatetimeIndex(train_X['earliesCreditLine']).month
test_X['earliesCreditLine_year'] = pd.DatetimeIndex(test_X['earliesCreditLine']).year
test_X['earliesCreditLine_month'] = pd.DatetimeIndex(test_X['earliesCreditLine']).month

#issueDate
train_X['issueDate_year'] = pd.DatetimeIndex(train_X['issueDate']).year
train_X['issueDate_month'] = pd.DatetimeIndex(train_X['issueDate']).month
train_X['issueDate_day'] = pd.DatetimeIndex(train_X['issueDate']).day

test_X['issueDate_year'] = pd.DatetimeIndex(test_X['issueDate']).year
test_X['issueDate_month'] = pd.DatetimeIndex(test_X['issueDate']).month
test_X['issueDate_day'] = pd.DatetimeIndex(test_X['issueDate']).day

In [16]:
#将 处理好的object类型特征删除掉
for col in train_X.columns:
    if train_X[col].dtype == 'object':
        train_X.drop(labels = [col], axis = 1, inplace =True)
        test_X.drop(labels = [col], axis = 1, inplace =True)

In [17]:
gc.collect()

0

In [18]:
train_X.head().T

,0,1,2,3,4
id,0.00,1.00,2.00,3.00,4.00
loanAmnt,35000.00,18000.00,12000.00,11000.00,3000.00
term,5.00,5.00,5.00,3.00,3.00
interestRate,19.52,18.49,16.99,7.26,12.99
installment,917.97,461.90,298.17,340.96,101.07
grade,4.00,3.00,3.00,0.00,2.00
subGrade,21.00,16.00,17.00,3.00,11.00
employmentTitle,320.00,219843.00,31698.00,46854.00,54.00
employmentLength,2.00,5.00,8.00,1.00,11.00
homeOwnership,2.00,0.00,0.00,1.00,1.00


In [19]:
sys.getrefcount(train_X)

2

### 其他category类型特征

# 特征编码

In [20]:
#类别型特征
cate_cols = ['id','term','grade','subGrade','employmentTitle','employmentLength',
            'homeOwnership','verificationStatus','purpose','postCode','regionCode',
            'initialListStatus','applicationType','title','policyCode',
            'earliesCreditLine_year','earliesCreditLine_month','issueDate_year',
             'issueDate_month','issueDate_day']
#连续型特征
num_cols = [x for x in train_X.columns if x not in cate_cols]

In [21]:
#term
train_X['term'].value_counts()

3    606902
5    193098
Name: term, dtype: int64

In [22]:
#grade
train_X['grade'].value_counts()

1    233690
2    227118
0    139661
3    119453
4     55661
5     19053
6      5364
Name: grade, dtype: int64

In [23]:
#subGrade
train_X['subGrade'].value_counts()

10    50763
8     49516
9     48965
7     48600
11    47068
12    44751
13    44272
6     44227
5     42382
14    40264
4     38045
3     30928
15    30538
16    26528
0     25909
17    23410
2     22655
1     22124
18    21139
19    17838
20    14064
21    12746
22    10925
23     9273
24     8653
25     5925
26     4340
27     3577
28     2859
29     2352
30     1759
31     1231
32      978
33      751
34      645
Name: subGrade, dtype: int64

In [24]:
#employmentLength
train_X['employmentLength'].value_counts()

1     262753
2      72358
10     64237
3      64152
0      52489
5      50102
4      47985
11     46799
6      37254
8      36192
7      35407
9      30272
Name: employmentLength, dtype: int64

In [25]:
#employmentTitle
"""从数值结果看，employmentTitle属于高基数类别特征"""
train_X['employmentTitle'].value_counts()

54.0        51149
38.0        12644
32.0        11543
184.0        6112
151.0        5193
            ...  
32737.0         1
56828.0         1
118913.0        1
208948.0        1
134854.0        1
Name: employmentTitle, Length: 248684, dtype: int64

In [26]:
#对employmentTitle进行均值编码
temp_df = train_X[~train_X['isDefault'].isnull()].copy()
temp_label = train_X['isDefault']
mean_enc = MeanEncoder(['employmentTitle'])
train_X = mean_enc.fit_transform(temp_df,temp_label)
del temp_df,temp_label
gc.collect()

0

In [27]:
#homeOwnership
train_X['homeOwnership'].value_counts()

0    395732
1    317660
2     86309
3       185
5        81
4        33
Name: homeOwnership, dtype: int64

In [28]:
# verificationStatus
train_X['verificationStatus'].value_counts()

1    309810
2    248968
0    241222
Name: verificationStatus, dtype: int64

In [29]:
#purpose
train_X['purpose'].value_counts()

0     464096
4     175433
2      52129
5      46276
3      17579
9       9238
1       9106
8       8657
10      5652
7       5373
6       4354
12      1363
11       554
13       190
Name: purpose, dtype: int64

In [30]:
#postCode
train_X['postCode'].value_counts()

134.0    8956
19.0     8611
51.0     8161
31.0     7337
4.0      7176
         ... 
886.0       1
884.0       1
839.0       1
899.0       1
935.0       1
Name: postCode, Length: 933, dtype: int64

In [31]:
#可以先将postCode进行mean编码尝试一下
temp_df = train_X[~train_X['isDefault'].isnull()].copy()
temp_label = train_X['isDefault']
mean_enc = MeanEncoder(['postCode'])
train_X = mean_enc.fit_transform(temp_df,temp_label)
del temp_df, temp_label
gc.collect()

0

In [32]:
#regionCode
train_X['regionCode'].value_counts()

8     116921
14     65768
13     65041
21     56671
2      30513
30     28634
0      27180
19     26198
3      25766
9      22902
7      22600
23     20919
10     19604
12     18543
26     18432
22     17722
18     17286
4      14177
11     12929
24     12776
32     12065
38     11982
36     11644
27     10521
17      9863
35      9669
5       9581
20      9124
43      7701
42      7267
15      6690
37      5998
45      5932
16      4325
28      4036
44      3963
33      3817
6       3496
39      2943
40      2287
31      2261
34      2136
25      2102
48      1880
41      1778
1       1624
29      1560
47      1213
49      1001
46       953
50         6
Name: regionCode, dtype: int64

In [33]:
#initialListStatus
train_X['initialListStatus'].value_counts()

0    466438
1    333562
Name: initialListStatus, dtype: int64

In [34]:
#applicationType
train_X['applicationType'].value_counts()

0    784586
1     15414
Name: applicationType, dtype: int64

In [35]:
#title
train_X['title'].value_counts()

0.0        393334
4.0        148211
5.0         44748
6.0         39742
3.0         14432
            ...  
59059.0         1
53052.0         1
42638.0         1
28481.0         1
33369.0         1
Name: title, Length: 39645, dtype: int64

In [36]:
temp_df = train_X[~train_X['isDefault'].isnull()].copy()
temp_label = train_X['isDefault']

mean_enc = MeanEncoder(['title'])
train_X = mean_enc.fit_transform(temp_df, temp_label)
del temp_df, temp_label
gc.collect()

0

In [37]:
#policyCode
train_X['policyCode'].value_counts()

1.0    800000
Name: policyCode, dtype: int64

In [38]:
#earliesCreditLine_year
train_X['earliesCreditLine_year'].value_counts()

2001    53194
2002    51060
2003    50649
2000    50624
2004    49280
        ...  
1953        5
1950        5
1954        5
1946        2
1944        1
Name: earliesCreditLine_year, Length: 68, dtype: int64

In [39]:
#earliesCreditLine_month
train_X['earliesCreditLine_month'].value_counts()

8     79208
9     77698
10    76726
11    70407
3     65035
12    64493
7     64440
5     62440
6     61858
4     61015
1     58605
2     58075
Name: earliesCreditLine_month, dtype: int64

In [40]:
# issueDate_year
train_X['issueDate_year'].value_counts()

2015    223387
2016    173907
2014    133087
2017    100736
2013     79943
2018     33415
2012     31895
2011     12924
2010      6861
2009      2781
2008       907
2007       157
Name: issueDate_year, dtype: int64

In [41]:
# issueDate_month
train_X['issueDate_month'].value_counts()

10    81035
7     76969
3     74774
11    67785
1     67280
8     66842
4     65147
5     62159
12    62121
2     62108
6     59689
9     54091
Name: issueDate_month, dtype: int64

In [42]:
# issueDate_day
train_X['issueDate_day'].value_counts()

1    800000
Name: issueDate_day, dtype: int64

In [43]:
cate_cols = ['term','grade', 'subGrade','employmentTitle', 'employmentLength']
num_cols = ['loanAmnt','interestRate', 'installment',  'homeOwnership', 'annualIncome',
          'dti', 'delinquency_2years', 'ficoRangeLow', 
       'revolBal', 'n1',  'n11', 'n14']

In [44]:
5 * len(num_cols) * 5

300

In [45]:
gc.collect()
sys.getrefcount(train_X)
train_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800000 entries, 0 to 799999
Data columns (total 56 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       800000 non-null  int64  
 1   loanAmnt                 800000 non-null  float64
 2   term                     800000 non-null  int64  
 3   interestRate             800000 non-null  float64
 4   installment              800000 non-null  float64
 5   grade                    800000 non-null  int32  
 6   subGrade                 800000 non-null  int32  
 7   employmentTitle          800000 non-null  float64
 8   employmentLength         800000 non-null  int32  
 9   homeOwnership            800000 non-null  int64  
 10  annualIncome             800000 non-null  float64
 11  verificationStatus       800000 non-null  int64  
 12  isDefault                800000 non-null  int64  
 13  purpose                  800000 non-null  int64  
 14  post

In [46]:
temp_df = []
for cate_i in cate_cols:
    print(cate_i,'\n')
    for num_i in num_cols:
        col_name = '{}_{}_'.format(cate_i, num_i)
        temp_df = train_X.groupby([cate_i], as_index=False)[num_i].agg(
            {col_name + 'max': 'max',
            col_name + 'min':'min',
            col_name + 'median': 'median',
            col_name + 'mean': 'mean',
            col_name + 'std' :'std',
            })
        train_X = train_X.merge(temp_df, on=cate_i,how='left')  
        del temp_df
        gc.collect()

term 

grade 

subGrade 

employmentTitle 

employmentLength 



In [47]:
#用numerical特征减去 经过category特征分组后的min,max,median,mean,std
for num_i in num_cols:
    for cate_i in cate_cols:
        for op_i in ['max','min','mean','median','std']:
            col_name = '{}_{}_{}'.format(cate_i,num_i,op_i)
            train_X[num_i + '-' + col_name] = train_X[num_i] - train_X[col_name]
    gc.collect()   

In [48]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800000 entries, 0 to 799999
Columns: 656 entries, id to n14-employmentLength_n14_std
dtypes: float64(619), int32(3), int64(34)
memory usage: 3.9 GB


In [49]:
gc.collect()
sys.getrefcount(train_X)

2

In [50]:
train_X = train_X.astype('float32')

In [51]:
gc.collect()
sys.getrefcount(train_X)
train_X.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 800000 entries, 0 to 799999
Columns: 656 entries, id to n14-employmentLength_n14_std
dtypes: float32(656)
memory usage: 2.0 GB


# 对category feature 进行COUNTS编码

In [52]:
#选择需要counts计数的类别
cate_cols.append('issueDate_year')

In [53]:
for col in cate_cols:
    train_X[col + '_count'] = train_X[col].map(train_X[col].value_counts())

# 从业务角度结合+-*/构建新特征

In [54]:
train_X['loanAmnt_interestRate_crosses'] = train_X['loanAmnt'] * train_X['interestRate']
train_X['loanAmnt_term_crosses'] = train_X['loanAmnt'] / train_X['term']
train_X['employmentLength_annualIncome_crosses'] = train_X['employmentLength'] * train_X['annualIncome']
train_X['pubRecBankruptcies_pubRec_crosses'] = train_X['pubRecBankruptcies'] / train_X['pubRec']
train_X['issueDate_year_earliesCreditLine_year_crosses'] = train_X['issueDate_year'] - train_X['earliesCreditLine_year']
train_X['installment_loanAmnt_crosses'] = train_X['installment'] / train_X['loanAmnt']
train_X['loanAmnt_totalAcc_crosses'] = train_X['loanAmnt'] / train_X['totalAcc']
train_X['loanAmnt_annualIncome_crosses'] = train_X['loanAmnt'] / train_X['annualIncome']
train_X['loanAmnt_interestRate_installment_crosses'] = train_X['loanAmnt'] * train_X['interestRate'] / train_X['installment'] 
train_X['loanAmnt_interestRate_annuallncome_crosses'] = train_X['loanAmnt'] * train_X['interestRate'] / train_X['annualIncome']
train_X['subGrade_term'] = train_X['subGrade'] * train_X['term']
train_X['grade_issueDate_year'] = train_X['grade'] * train_X['issueDate_year']
train_X['subGrade_issueDate_year'] = train_X['subGrade'] * train_X['issueDate_year']
train_X['grade_term'] = train_X['grade'] * train_X['term']
train_X['subGrade_n2'] = train_X['subGrade'] * train_X['n2']
train_X['subGrade_dti'] = train_X['subGrade'] * train_X['dti']
train_X['term_interestRate'] = train_X['term'] * train_X['interestRate']
train_X['grade_loanAmnt'] = train_X['grade'] * train_X['loanAmnt']
train_X['ficoRangeHigh_ficoRangeLow_crosses'] = train_X['ficoRangeHigh'] - train_X['ficoRangeLow']
train_X['ficoRangeHigh_ficoRangeLow_2_crosses'] =(train_X['ficoRangeHigh'] - train_X['ficoRangeLow'])/ 2
train_X['installment_annualIncome_crosses'] = train_X['installment'] * 12 /  train_X['annualIncome']

In [55]:
gc.collect()
sys.getrefcount(train_X)
train_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800000 entries, 0 to 799999
Columns: 683 entries, id to installment_annualIncome_crosses
dtypes: float32(677), int64(6)
memory usage: 2.1 GB


In [56]:
#将数据集中正负无穷的值替换为空值
train_X.replace([np.inf, -np.inf], np.nan, inplace = True)

In [57]:
#暂时先填充空值
for col in train_X.columns[train_X.isnull().sum() > 0].to_list():
    imputer = SimpleImputer(strategy='constant', fill_value= -999)
    train_X[col] = imputer.fit_transform(train_X[[col]])
    del imputer
    gc.collect()

In [58]:
gc.collect()
sys.getrefcount(train_X)
train_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800000 entries, 0 to 799999
Columns: 683 entries, id to installment_annualIncome_crosses
dtypes: float64(677), int64(6)
memory usage: 4.1 GB


In [59]:
"""以上683个特征，包含min，max,median,mean,std, count统计
以及业务特征的+-*/，
维度过高，会导致模型过拟合，现在进行特征筛选

"""

'以上683个特征，包含min，max,median,mean,std, count统计\n以及业务特征的+-*/，\n维度过高，会导致模型过拟合，现在进行特征筛选\n\n'

# 特征筛选

## 过滤法——方差过滤

In [60]:
train_y = train_X['isDefault']
train_X.drop(labels = ['isDefault'], axis = 1, inplace = True)

In [61]:
var_select = VarianceThreshold(0.8)
var_res = var_select.fit_transform(train_X)
train_X = pd.DataFrame(data = var_res,columns=var_select.get_feature_names_out())

In [62]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Columns: 518 entries, id to installment_annualIncome_crosses
dtypes: float64(518)
memory usage: 3.1 GB


In [63]:
train_X = train_X.astype('float32')
train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Columns: 518 entries, id to installment_annualIncome_crosses
dtypes: float32(518)
memory usage: 1.5 GB


In [64]:
train_X['id']

0              0.0
1              1.0
2              2.0
3              3.0
4              4.0
            ...   
799995    799995.0
799996    799996.0
799997    799997.0
799998    799998.0
799999    799999.0
Name: id, Length: 800000, dtype: float32

In [65]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Columns: 518 entries, id to installment_annualIncome_crosses
dtypes: float32(518)
memory usage: 1.5 GB


In [66]:
train_X.to_pickle("./v12.pkl")

## 过滤法——相关性

In [67]:
"""过于耗时"""

'过于耗时'

In [ ]:
temp_df = pd.read_csv("./v13.csv")

In [ ]:
temp_df.info()

In [ ]:
#划分测试集和验证集
train_X,val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=0.3,random_state= seed_state)

In [ ]:
"""利用optuna做参数调优"""
def objective(trial):
    
    params = {
            'n_estimators': trial.suggest_int('n_estimators', 1000,10000,2000),
            'max_depth' : trial.suggest_int('max_depth', 5, 25, 1),
            'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.3, step=0.02)
    }


    model = xgboost.XGBClassifier(**params,
                             subsample = 0.8,
                             reg_alpha = 3,
                             reg_lambda = 1,
                             tree_method = 'gpu_hist',
                             random_state = seed_state
                             )
    model.fit(train_X,train_y,early_stopping_rounds=100, verbose=0,eval_set=[(val_X,val_y)])
    
    
    preds_prob = model.predict_proba(val_X)
    
    return roc_auc_score(val_y, preds_prob[:,1])

In [ ]:
start_time = time.time()
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials = 50)
end_time = time.time()
study.best_params
study.best_value

In [ ]:
end_time - start_time

In [ ]:
print(study.best_params)
print(study.best_value)

In [ ]:
feature_names = list(train_X.columns)

#包外验证
df_oof = train_X[['id']].copy()
df_oof['label'] = train_y
df_oof['prob'] = 0

#存放测试结果
# prediction = test_X[['id']]
# prediction['prob'] = 0

#重要特征列表
df_importance_list = []

model = xgboost.XGBClassifier(
                            n_estimators = 7000,
                            n_jobs = -1,
                            max_depth = 6,
                            learning_rate = 0.01,
                            subsample = 0.8,
                            tree_method = 'gpu_hist',
                            gamma = 0.3,
                            reg_alpha = 3,
                            reg_lambda = 1,
                            random_state = seed_state)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed_state)

for fold_id,(train_idx, val_idx) in enumerate(kfold.split(train_X, train_y)):
    train_X_kfold = train_X.iloc[train_idx]
    train_y_kfold = train_y.iloc[train_idx]
    val_X_kfold = train_X.iloc[val_idx]
    val_y_kfold = train_y.iloc[val_idx]
    
    print('\nFold_{} Training ========================\n'.format(
        fold_id + 1))
    
    xgb_model = model.fit(train_X_kfold,
                          train_y_kfold,
                          eval_set=[(train_X_kfold, train_y_kfold), (val_X_kfold, val_y_kfold)],
                          verbose=50,
                          eval_metric='auc',
                          early_stopping_rounds=50)
    
    pred_val = xgb_model.predict_proba(val_X_kfold)[:, 1]
    df_oof.loc[val_idx, 'prob'] = pred_val
    
    
#     pred_test = xgb_model.predict_proba(test_X)[:, 1]
#     prediction['prob'] += pred_test / kfold.n_splits

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': xgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del xgb_model, pred_val, train_X_kfold, train_y_kfold, val_X_kfold, val_y_kfold
    gc.collect()
    
    
print(model.best_score)

In [ ]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].mean().sort_values(ascending = False).reset_index()

In [ ]:
df_importance.to_csv("./feature_importance__.csv")

In [2]:
df = pd.DataFrame(data=[1,2,3,4,5,6,np.nan,np.nan])


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       6 non-null      float64
dtypes: float64(1)
memory usage: 192.0 bytes
